In [7]:
import tensorflow as tf
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.applications import VGG19
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dropout,Input,Flatten,Dense,MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator  # Data Augumentation

In [8]:
batchsize=100

In [18]:

train_datagen= ImageDataGenerator(rescale=1./255, rotation_range=0.2,shear_range=0.2,
    zoom_range=0.2,width_shift_range=0.2,
    height_shift_range=0.2, validation_split=0.2)

train_data= train_datagen.flow_from_directory(r'C:\Users\vigna\Desktop\mini project\prepared_data\train',
                                target_size=(80,80),batch_size=batchsize,class_mode='categorical',subset='training' )

validation_data= train_datagen.flow_from_directory(r'C:\Users\vigna\Desktop\mini project\prepared_data\train',
                                target_size=(80,80),batch_size=batchsize,class_mode='categorical', subset='validation')

Found 61379 images belonging to 2 classes.
Found 15343 images belonging to 2 classes.


In [19]:
test_datagen = ImageDataGenerator(rescale=1./255)

test_data = test_datagen.flow_from_directory(r'C:\Users\vigna\Desktop\mini project\prepared_data\test',
                                target_size=(80,80),batch_size=batchsize,class_mode='categorical')

Found 8176 images belonging to 2 classes.


In [20]:
bmodel = InceptionV3(include_top=False, weights='imagenet', input_tensor=Input(shape=(80,80,3)))
hmodel = bmodel.output
hmodel = Flatten()(hmodel)
hmodel = Dense(64, activation='relu')(hmodel)
hmodel = Dropout(0.5)(hmodel)
hmodel = Dense(2,activation= 'softmax')(hmodel)

model = Model(inputs=bmodel.input, outputs= hmodel)
for layer in bmodel.layers:
    layer.trainable = False

In [21]:
bmodel.summary()

Model: "inception_v3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_6 (InputLayer)           [(None, 80, 80, 3)]  0           []                               
                                                                                                  
 conv2d_94 (Conv2D)             (None, 39, 39, 32)   864         ['input_6[0][0]']                
                                                                                                  
 batch_normalization_94 (BatchN  (None, 39, 39, 32)  96          ['conv2d_94[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 activation_94 (Activation)     (None, 39, 39, 32)   0           ['batch_normalization_

In [22]:
from tensorflow.keras.callbacks import ModelCheckpoint,EarlyStopping, ReduceLROnPlateau

In [23]:

checkpoint = ModelCheckpoint(r'C:\Users\vigna\Desktop\mini project\model\model_ensemble.hdf5',
                            monitor='val_loss',save_best_only=True,verbose=3)

learning_rate = ReduceLROnPlateau(monitor= 'val_loss', patience=3, verbose= 3 )

callbacks=[checkpoint,learning_rate]

In [24]:
model.compile(optimizer='Adam', loss='categorical_crossentropy',metrics=['accuracy'])
history1=model.fit(train_data,
                   validation_data=validation_data,
                   callbacks=callbacks,
                    epochs=50)

Epoch 1/50
614/614 [==============================] - ETA: 0s - loss: 0.2114 - accuracy: 0.9161
Epoch 1: val_loss improved from inf to 0.19900, saving model to C:\Users\vigna\Desktop\mini project\model\model_ensemble.hdf5
614/614 [==============================] - 215s 345ms/step - loss: 0.2114 - accuracy: 0.9161 - val_loss: 0.1990 - val_accuracy: 0.9138 - lr: 0.0010
Epoch 2/50
614/614 [==============================] - ETA: 0s - loss: 0.1700 - accuracy: 0.9349
Epoch 2: val_loss did not improve from 0.19900
614/614 [==============================] - 178s 290ms/step - loss: 0.1700 - accuracy: 0.9349 - val_loss: 0.2064 - val_accuracy: 0.9092 - lr: 0.0010
Epoch 3/50
614/614 [==============================] - ETA: 0s - loss: 0.1619 - accuracy: 0.9382
Epoch 3: val_loss improved from 0.19900 to 0.19308, saving model to C:\Users\vigna\Desktop\mini project\model\model_ensemble.hdf5
614/614 [==============================] - 180s 293ms/step - loss: 0.1619 - accuracy: 0.9382 - val_loss: 0.1931 -

In [25]:
bmodel1 = VGG19(include_top=False, weights='imagenet', input_tensor=Input(shape=(80,80,3)))
hmodel1 = bmodel1.output
hmodel1 = Flatten()(hmodel1)
hmodel1 = Dense(64, activation='relu')(hmodel1)
hmodel1 = Dropout(0.5)(hmodel1)
hmodel1 = Dense(2,activation= 'softmax')(hmodel1)

model1 = Model(inputs=bmodel1.input, outputs= hmodel1)
for layer in bmodel1.layers:
    layer.trainable = False

In [26]:
checkpoint1 = ModelCheckpoint(r'C:\Users\vigna\Desktop\mini project\model\model_vgg19_ensemble.hdf5',
                            monitor='val_loss',save_best_only=True,verbose=3)
learning_rate1 = ReduceLROnPlateau(monitor= 'val_loss', patience=3, verbose= 3, )

callbacks1=[checkpoint1,learning_rate1]

In [27]:
model1.compile(optimizer='Adam', loss='categorical_crossentropy',metrics=['accuracy'])
history2=model1.fit(train_data,
                   validation_data=validation_data,
                   callbacks=callbacks1,
                    epochs=20)


Epoch 1/20
614/614 [==============================] - ETA: 0s - loss: 0.4528 - accuracy: 0.7826
Epoch 1: val_loss improved from inf to 0.44804, saving model to C:\Users\vigna\Desktop\mini project\model\model_vgg19_ensemble.hdf5
614/614 [==============================] - 673s 1s/step - loss: 0.4528 - accuracy: 0.7826 - val_loss: 0.4480 - val_accuracy: 0.7981 - lr: 0.0010
Epoch 2/20
614/614 [==============================] - ETA: 0s - loss: 0.3437 - accuracy: 0.8529
Epoch 2: val_loss improved from 0.44804 to 0.44542, saving model to C:\Users\vigna\Desktop\mini project\model\model_vgg19_ensemble.hdf5
614/614 [==============================] - 832s 1s/step - loss: 0.3437 - accuracy: 0.8529 - val_loss: 0.4454 - val_accuracy: 0.7740 - lr: 0.0010
Epoch 3/20
614/614 [==============================] - ETA: 0s - loss: 0.3131 - accuracy: 0.8682
Epoch 3: val_loss improved from 0.44542 to 0.42827, saving model to C:\Users\vigna\Desktop\mini project\model\model_vgg19_ensemble.hdf5
614/614 [=========

In [28]:
bmode2 = ResNet50(include_top=False, weights='imagenet', input_tensor=Input(shape=(80,80,3)))
hmode2 = bmode2.output
hmode2 = Flatten()(hmode2)
hmode2 = Dense(64, activation='relu')(hmode2)
hmode2 = Dropout(0.5)(hmode2)
hmode2 = Dense(2,activation= 'softmax')(hmode2)
model3 = Model(inputs=bmode2.input, outputs= hmode2)
for layer in bmode2.layers:
    layer.trainable = False

In [29]:
checkpoint2 = ModelCheckpoint(r'C:\Users\vigna\Desktop\mini project\model\model_ensemble2.hdf5',
                            monitor='val_loss',save_best_only=True,verbose=3)

learning_rate2 = ReduceLROnPlateau(monitor= 'val_loss', patience=3, verbose= 3 )

callbacks2=[checkpoint2,learning_rate2]

In [30]:
model3.compile(optimizer='Adam', loss='categorical_crossentropy',metrics=['accuracy'])
history_=model3.fit_generator(train_data,steps_per_epoch=train_data.samples//batchsize,
                   validation_data=validation_data,
                   validation_steps=validation_data.samples//batchsize,
                   callbacks=callbacks2,
                    epochs=50)

Epoch 1/50


C:\Users\vigna\AppData\Local\Temp\ipykernel_26384\837215024.py:2: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history_=model3.fit_generator(train_data,steps_per_epoch=train_data.samples//batchsize,


613/613 [==============================] - ETA: 0s - loss: 0.6948 - accuracy: 0.5066
Epoch 1: val_loss improved from inf to 0.69303, saving model to C:\Users\vigna\Desktop\mini project\model\model_ensemble2.hdf5
613/613 [==============================] - 387s 629ms/step - loss: 0.6948 - accuracy: 0.5066 - val_loss: 0.6930 - val_accuracy: 0.5076 - lr: 0.0010
Epoch 2/50
613/613 [==============================] - ETA: 0s - loss: 0.6931 - accuracy: 0.5076
Epoch 2: val_loss did not improve from 0.69303
613/613 [==============================] - 388s 633ms/step - loss: 0.6931 - accuracy: 0.5076 - val_loss: 0.6930 - val_accuracy: 0.5078 - lr: 0.0010
Epoch 3/50
613/613 [==============================] - ETA: 0s - loss: 0.6931 - accuracy: 0.5077
Epoch 3: val_loss did not improve from 0.69303
613/613 [==============================] - 401s 654ms/step - loss: 0.6931 - accuracy: 0.5077 - val_loss: 0.6930 - val_accuracy: 0.5076 - lr: 0.0010
Epoch 4/50
613/613 [==============================] - ETA:

In [31]:
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Input, Average

mode = load_model(r'C:\Users\vigna\Desktop\mini project\model\model_ensemble.hdf5')
model_1 = Model(inputs=model.inputs,
                outputs=model.outputs,
                name='name_of_model_1') 
model_2 = load_model(r'C:\Users\vigna\Desktop\mini project\model\model_vgg19_ensemble.hdf5')
model_2 = Model(inputs=model1.inputs,
                outputs=model1.outputs,
                name='name_of_model_2')
model_3 = load_model(r'C:\Users\vigna\Desktop\mini project\model\model_ensemble2.hdf5')
model_3 = Model(inputs=model3.inputs,
                outputs=model3.outputs,
                name='name_of_model_3')
models = [model_1, model_2,model_3]
model_input = Input(shape=(80, 80, 3))
model_outputs = [model(model_input) for model in models]
ensemble_output = Average()(model_outputs)
ensemble_model = Model(inputs=model_input, outputs=ensemble_output, name='ensemble')

In [32]:
checkpointe = ModelCheckpoint(r'C:\Users\vigna\Desktop\mini project\model\model_ensemblefin.hdf5',
                            monitor='val_loss',save_best_only=True,verbose=3)

learning_ratee = ReduceLROnPlateau(monitor= 'val_loss', patience=3, verbose= 3 )

callbackse=[checkpointe,learning_ratee]

In [33]:
ensemble_model.compile(optimizer='Adam', loss='categorical_crossentropy',metrics=['accuracy'])
history3=ensemble_model.fit(train_data,
                   validation_data=validation_data,
                   callbacks=callbackse,
                    epochs=50)


Epoch 1/50
614/614 [==============================] - ETA: 0s - loss: 0.2850 - accuracy: 0.9449
Epoch 1: val_loss improved from inf to 0.31247, saving model to C:\Users\vigna\Desktop\mini project\model\model_ensemblefin.hdf5
614/614 [==============================] - 1066s 2s/step - loss: 0.2850 - accuracy: 0.9449 - val_loss: 0.3125 - val_accuracy: 0.9317 - lr: 0.0010
Epoch 2/50
614/614 [==============================] - ETA: 0s - loss: 0.2815 - accuracy: 0.9451
Epoch 2: val_loss did not improve from 0.31247
614/614 [==============================] - 1057s 2s/step - loss: 0.2815 - accuracy: 0.9451 - val_loss: 0.3245 - val_accuracy: 0.9181 - lr: 0.0010
Epoch 3/50
614/614 [==============================] - ETA: 0s - loss: 0.2814 - accuracy: 0.9425
Epoch 3: val_loss did not improve from 0.31247
614/614 [==============================] - 1112s 2s/step - loss: 0.2814 - accuracy: 0.9425 - val_loss: 0.3190 - val_accuracy: 0.9161 - lr: 0.0010
Epoch 4/50
614/614 [==============================]

In [35]:
acc_tr, loss_tr = ensemble_model.evaluate_generator(train_data)
print(acc_tr)
print(loss_tr)

C:\Users\vigna\AppData\Local\Temp\ipykernel_26384\1716953310.py:1: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  acc_tr, loss_tr = ensemble_model.evaluate_generator(train_data)


0.2581940293312073
0.9523941278457642


In [36]:

acc_test, loss_test = ensemble_model.evaluate_generator(test_data)
print(acc_test)
print(loss_test)

C:\Users\vigna\AppData\Local\Temp\ipykernel_26384\3541006010.py:1: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  acc_test, loss_test = ensemble_model.evaluate_generator(test_data)


0.3334234058856964
0.9085127115249634
